In [167]:
import os
import pandas as pd

In [168]:
cwd = os.getcwd()
df_syn_pop = pd.read_csv(os.path.join(os.path.dirname(cwd), 'output', 'adjusted_synthetic_persons.csv'))
df_syn_hh = pd.read_csv(os.path.join(os.path.dirname(cwd), 'output', 'adjusted_synthetic_households.csv'))

In [170]:
# Attach original seed person/household records
df_seed_pop = pd.read_csv(r'R:\e2projects_two\SyntheticPopulation_2018\populationsim_inputs\data\seed_persons.csv')
df_seed_hh = pd.read_csv(r'R:\e2projects_two\SyntheticPopulation_2018\populationsim_inputs\data\seed_households.csv')

In [171]:
# Person Records
df_new_pop = df_syn_pop.merge(df_seed_pop[['ESR','PINCP','SCHL','SCHG','WKHP','RELP','hhnum','SPORDER']],
                left_on=['hh_id','per_num'], right_on=['hhnum','SPORDER'])
df_new_pop.columns = [i.lower() for i in df_new_pop.columns]

In [172]:
len(df_syn_pop) == len(df_new_pop)

True

In [173]:
# Household Records
df_new_hh = df_syn_hh.merge(df_seed_hh[['hhnum','TEN','PUMA','NP']], left_on=['hh_id','PUMA','NP'], 
                            right_on=['hhnum','PUMA','NP'], how='left')

In [174]:
len(df_new_hh) == len(df_syn_hh)

True

In [175]:
block_group_table = pd.read_csv(r'R:\e2projects_two\2018_base_year\synthetic_population\census_block_groups.csv')

In [176]:
# Drop leading zeros to match format of synthetic population
block_group_table['state'] = block_group_table['census_2010_block_group_id'].apply(lambda x: int(str(x)[0:2]))
block_group_table['county'] = block_group_table['census_2010_block_group_id'].apply(lambda x: int(str(x)[3:5]))
block_group_table['tract'] = block_group_table['census_2010_block_group_id'].apply(lambda x: int(str(x)[5:11]))
block_group_table['block_group'] = block_group_table['census_2010_block_group_id'].apply(lambda x: int(str(x)[11:]))

block_group_table['newid'] = block_group_table['state'].astype('str') + block_group_table['county'].astype('str') + block_group_table['tract'].astype('str') + block_group_table['block_group'].astype('str')
# block_group_table['newid'] = block_group_table['newid'].astype('int')

In [177]:
df_new_hh['newid'] = df_new_hh['block_group_id'].astype('str')
df = block_group_table.merge(df_new_hh, on='newid')

In [193]:
df.to_csv(r'C:\Users\bnichols\Documents\populationsim\psrc\output\finalized\adjusted_synthetic_households.csv', index=False)

In [178]:
df_new_pop['newid'] = df_new_pop['block_group_id'].astype('str')
df_person = block_group_table.merge(df_new_pop, on='newid')

In [179]:
df_person['new_per_num'] = df_person['new_per_num'].fillna(df_person['sporder'])

In [194]:
df_person.to_csv(r'C:\Users\bnichols\Documents\populationsim\psrc\output\finalized\adjusted_synthetic_persons.csv', index=False)